In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import sys
import wave

def zapus(n,fl):
    RATE = 44100
    CHUNK = 512*16
    filename=fl
    znach1 = wave.open(filename,"r")
    znach1_16 = znach1.readframes(-1)
    znach2 = np.frombuffer(znach1_16, dtype='int16')
    print(len(znach2))
    audio_file = wave.open(filename)
    CHANNELS2 = audio_file.getnchannels()
    
    text=open('spilne2_'+str(n)+'.txt','w')  
    xf = np.linspace(0, CHANNELS2*RATE, CHUNK)  #від 0 до RATE розмістити CHUNK точок 
    for k in range(int(len(znach2)/CHUNK)):
        yf = fft(znach2[CHUNK*k:CHUNK*(k+1)])
        yf2=[]
        for k in range(len(yf)):
            yf2.append(abs(yf[k])/(0.5 * CHUNK))
        text.write(str(round(xf[yf2.index(max(yf2[0:int(len(yf2)/4)]))]))+' ')
    text.close()
    return ''

def korel(x,y):
    xs=sum(x)/len(x)
    ys=sum(y)/len(y)
    dx,dy,dx2,dy2,dxy=[],[],0,0,0
    for k in range(len(x)):
        dx=dx+[x[k]-xs]
        dy=dy+[y[k]-ys]
        dxy=dxy+dx[-1]*dy[-1]
        dx2=dx2+dx[-1]**2
        dy2=dy2+dy[-1]**2
#     print(dxy,dx[0:20],dy[0:20])
    if dy2==0:
        dy2=1
    if dx2==0:
        dx2=1
    if dxy==0:
        dxy=1
    return dxy/(dx2*dy2)**(0.5)

def max_r(x,y):
    r=500
    zamina=[]
    sum_x,sum_y=0,0
    for k in range(len(x)):
        if abs(x[k]-y[k])>r:
            zamina.append(k)
        else:
            sum_x=sum_x+x[k]
            sum_y=sum_y+y[k]
    if len(zamina)>0:
        xs=sum_x/(len(x)-len(zamina))
        ys=sum_y/(len(y)-len(zamina))
        x1,x2=x.copy(),x.copy()
        y1,y2=y.copy(),y.copy()
        for k in range(len(zamina)):
            x1[zamina[k]]=xs
            y2[zamina[k]]=ys
        return (x1,y1,x2,y2,len(zamina))
    else:
        return 0
        

file=['sine.wav','Square.wav']
zapus(1,file[0])
zapus(2,file[1])

text1=open('spilne2_1.txt','r')
text2=open('spilne2_2.txt','r')
t1=1
t2=2
dt=512
f=44100
n1=int(t1*f/dt)
n2=int(t2*f/dt)
s1=text1.read()
s2=text2.read()
ch1=list(map(int, s1.split( )))
ch2=list(map(int, s2.split( )))
n1,n2=len(ch1),len(ch2)
text1.close()
text2.close()
v=[]
e=0
ch1=ch1[:n1]
ch2=ch2[:n2]
for i in range(n1):
    for j in range(n2):
        k1,k2=i,j
        while k1<n1 and k2<n2 and abs(ch1[k1]-ch2[k2])<=e:
            k1,k2=k1+1,k2+1
        v.append([i,j,k1-i])
v1=sorted(v,key=lambda x: x[2], reverse = True)

if v1[0][0]>v1[0][1]:
    ch_1=ch1[v1[0][0]-v1[0][1]:len(ch1)]
    ch_2=ch2.copy()
else:
    ch_2=ch2[v1[0][1]-v1[0][0]:len(ch2)]
    ch_1=ch1.copy()
ch_1=ch_1[0:min(len(ch_1),len(ch_2))]
ch_2=ch_2[0:len(ch_1)]

mas=max_r(ch_1.copy(),ch_2.copy())
if mas!=0:
    ch1_1,ch2_1,ch1_2,ch2_2,z=mas
    print(korel(ch1_1,ch2_1))
    print(korel(ch1_2,ch2_2))

    print('z=',z)

# k = np.corrcoef(ch_1,ch_2)
# print(k[0,1])
print(korel(ch_1,ch_2))

87552
87552
1.0
